In [1]:
from google.colab import drive
import pandas as pd
from collections import defaultdict
import numpy as np
import ast
import random
from datetime import datetime, timedelta
import json
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

In [98]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/emails_cleaned.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,From,Date,Subject,Body,Recipients,Domains,Label
0,vince.kaminski@enron.com,03-01-2001 07:07:00,L Xu,"Norma, Happy New Year. Look forward to working...",['norma.villarreal@enron.com'],[],1
1,kay.mann@enron.com,19-10-2000 08:01:00,Weekly Power Meetings With Sheila Tweed,FYI.,"['jeffrey.hodge@enron.com', 'barbara.gray@enro...",[],1
2,john.lavorato@enron.com,25-09-2000 03:12:00,Re: John Forney,Tim There is absolutely a spot for John. Maybe...,['tim.belden@enron.com'],[],1
3,rosalee.fleming@enron.com,21-08-2001 12:16:01,Management Committee Meetings,Please attend the Management Committee meeting...,"['james.derrick@enron.com', 'kevin.hannon@enro...",[],1
4,sherri.sera@enron.com,05-04-2000 03:32:00,More tickets for Friday night...,"Greg, if there are two more tickets available ...",['greg.grissom@enron.com'],[],1


In [99]:
df.sample(n=5)

,From,Date,Subject,Body,Recipients,Domains,Label
24674,eric.bass@enron.com,15-10-2001 12:14:59,Ski trip????,Are we going to plan one for Dec?,"['daphneco64@alltel.net', 'lwbthemarine@alltel...",[],1
347553,robert.bruce@enron.com,11-08-2000 07:09:00,ATOM registration,"According to Sidley & Austin, the CFTC has reg...","['sara.shackleton@enron.com', 'gary.hickerson@...",['livestocknutrition.com'],1
379688,janine.migden@enron.com,20-06-2000 06:57:00,Fwd: NCLC Energy: Electronic Signature; Federa...,This law appears to supercede state laws and a...,"['ginger.dernehl@enron.com', 'richard.shapiro@...",['lists.nclc.org'],1
28588,steve.hooser@enron.com,27-09-2000 10:06:00,Re: Coogler's Response,"Barbara, I had a chance to look at this again ...",['barbara.gray@enron.com'],[],1
125702,rod.hayslett@enron.com,12-12-2000 03:33:00,AMEX Gift Certificate,"Hi Felicia, I would like to have one (1) AMEX ...",['felicia.beal@enron.com'],[],1


In [100]:
df = df.drop(columns=['Label'])
df['Label'] = 0

In [101]:
df.head()

,From,Date,Subject,Body,Recipients,Domains,Label
0,vince.kaminski@enron.com,03-01-2001 07:07:00,L Xu,"Norma, Happy New Year. Look forward to working...",['norma.villarreal@enron.com'],[],0
1,kay.mann@enron.com,19-10-2000 08:01:00,Weekly Power Meetings With Sheila Tweed,FYI.,"['jeffrey.hodge@enron.com', 'barbara.gray@enro...",[],0
2,john.lavorato@enron.com,25-09-2000 03:12:00,Re: John Forney,Tim There is absolutely a spot for John. Maybe...,['tim.belden@enron.com'],[],0
3,rosalee.fleming@enron.com,21-08-2001 12:16:01,Management Committee Meetings,Please attend the Management Committee meeting...,"['james.derrick@enron.com', 'kevin.hannon@enro...",[],0
4,sherri.sera@enron.com,05-04-2000 03:32:00,More tickets for Friday night...,"Greg, if there are two more tickets available ...",['greg.grissom@enron.com'],[],0


In [102]:
print("Start date:", df['Date'].min())
print("End date:", df['Date'].max())

Start date: 01-01-2000 06:36:00
End date: 31-12-2001 23:56:37


In [103]:
earliest_date = df['Date'].min()
df[df['Date'] == earliest_date]

,From,Date,Subject,Body,Recipients,Domains,Label
56440,sally.beck@enron.com,01-01-2000 06:36:00,Happy New Year - No Y2K Fear!,We are wrapping up several hours in the office...,['fernley.dyson@enron.com'],[],0
224369,sally.beck@enron.com,01-01-2000 06:36:00,Happy New Year - No Y2K Fear!,We are wrapping up several hours in the office...,['fernley.dyson@enron.com'],[],0
268070,sally.beck@enron.com,01-01-2000 06:36:00,Happy New Year - No Y2K Fear!,We are wrapping up several hours in the office...,['fernley.dyson@enron.com'],[],0
276859,sally.beck@enron.com,01-01-2000 06:36:00,Happy New Year - No Y2K Fear!,We are wrapping up several hours in the office...,['fernley.dyson@enron.com'],[],0


In [104]:
df.isnull().sum()

,0
From,0
Date,0
Subject,0
Body,1
Recipients,0
Domains,0
Label,0


In [105]:
df = df.dropna()

In [106]:
df.count()

,0
From,382722
Date,382722
Subject,382722
Body,382722
Recipients,382722
Domains,382722
Label,382722


In [107]:
empty_count = df['Domains'].apply(lambda x: x.strip() == '[]').sum()
print(empty_count)

351227


In [108]:
df = df[df['Domains'].apply(lambda x: x.strip() != '[]')]

In [109]:
df.count()

,0
From,31495
Date,31495
Subject,31495
Body,31495
Recipients,31495
Domains,31495
Label,31495


In [110]:
df.head()

,From,Date,Subject,Body,Recipients,Domains,Label
5,kenny.soignet@enron.com,06-12-2000 05:06:00,AGA for 12/1/00 is -73,AGA for 12/1/00 is -73 Website information: ht...,"['kenny.soignet@enron.com', 'jean.mrha@enron.c...",['gasfundy.corp.enron.com'],0
9,announcements.enron@enron.com,22-02-2002 07:30:36,Outlook Web Access Outage,Enron's external Outlook Web Access (OWA) (htt...,['dl-ga-all_domestic@enron.com'],['mail.enron.com'],0
11,perfmgmt@enron.com,06-11-2000 14:59:00,Year End 2000 Performance Feedback,NOTE: YOU WILL RECEIVE THIS MESSAGE EACH TIME ...,['jane.m.tholt@enron.com'],['pep.corp.enron.com'],0
22,jad.doan@enron.com,29-10-2001 18:11:54,TRV Notification: (CE P/L - 10/29/2001),"The report named: CE P/L , published as of 10/...","['brad.jones@enron.com', 'lisa.kinsey@enron.co...",['trv.corp.enron.com'],0
54,enron.announcements@enron.com,05-06-2001 11:46:00,Managers Training,Attention supervisors and managers! =01; =01; ...,['all.houston@enron.com'],['corptraining.com'],0


In [111]:
df = df.reset_index(drop=True)

In [112]:
df.head()

,From,Date,Subject,Body,Recipients,Domains,Label
0,kenny.soignet@enron.com,06-12-2000 05:06:00,AGA for 12/1/00 is -73,AGA for 12/1/00 is -73 Website information: ht...,"['kenny.soignet@enron.com', 'jean.mrha@enron.c...",['gasfundy.corp.enron.com'],0
1,announcements.enron@enron.com,22-02-2002 07:30:36,Outlook Web Access Outage,Enron's external Outlook Web Access (OWA) (htt...,['dl-ga-all_domestic@enron.com'],['mail.enron.com'],0
2,perfmgmt@enron.com,06-11-2000 14:59:00,Year End 2000 Performance Feedback,NOTE: YOU WILL RECEIVE THIS MESSAGE EACH TIME ...,['jane.m.tholt@enron.com'],['pep.corp.enron.com'],0
3,jad.doan@enron.com,29-10-2001 18:11:54,TRV Notification: (CE P/L - 10/29/2001),"The report named: CE P/L , published as of 10/...","['brad.jones@enron.com', 'lisa.kinsey@enron.co...",['trv.corp.enron.com'],0
4,enron.announcements@enron.com,05-06-2001 11:46:00,Managers Training,Attention supervisors and managers! =01; =01; ...,['all.houston@enron.com'],['corptraining.com'],0


In [113]:
import pandas as pd
import re

def extract_emails(cell):
    if not cell:
        return []

    # Extract full email addresses using regex
    emails = re.findall(r'[\w\.-]+@[\w\.-]+', str(cell))

    return emails

# Apply the function
df['Recipients'] = df['Recipients'].apply(extract_emails)

In [114]:
df.head()

,From,Date,Subject,Body,Recipients,Domains,Label
0,kenny.soignet@enron.com,06-12-2000 05:06:00,AGA for 12/1/00 is -73,AGA for 12/1/00 is -73 Website information: ht...,"[kenny.soignet@enron.com, jean.mrha@enron.com,...",['gasfundy.corp.enron.com'],0
1,announcements.enron@enron.com,22-02-2002 07:30:36,Outlook Web Access Outage,Enron's external Outlook Web Access (OWA) (htt...,[dl-ga-all_domestic@enron.com],['mail.enron.com'],0
2,perfmgmt@enron.com,06-11-2000 14:59:00,Year End 2000 Performance Feedback,NOTE: YOU WILL RECEIVE THIS MESSAGE EACH TIME ...,[jane.m.tholt@enron.com],['pep.corp.enron.com'],0
3,jad.doan@enron.com,29-10-2001 18:11:54,TRV Notification: (CE P/L - 10/29/2001),"The report named: CE P/L , published as of 10/...","[brad.jones@enron.com, lisa.kinsey@enron.com, ...",['trv.corp.enron.com'],0
4,enron.announcements@enron.com,05-06-2001 11:46:00,Managers Training,Attention supervisors and managers! =01; =01; ...,[all.houston@enron.com],['corptraining.com'],0


In [115]:
import ast

def fix_domains(cell):
    if not cell or pd.isna(cell):
        return []

    # Try to evaluate string to list
    try:
        domains = ast.literal_eval(cell) if isinstance(cell, str) else cell
    except:
        return []

    # Clean each domain string
    return [d.strip("'\"") for d in domains if isinstance(d, str)]

# Apply
df['Domains'] = df['Domains'].apply(fix_domains)

In [116]:
df.head()

,From,Date,Subject,Body,Recipients,Domains,Label
0,kenny.soignet@enron.com,06-12-2000 05:06:00,AGA for 12/1/00 is -73,AGA for 12/1/00 is -73 Website information: ht...,"[kenny.soignet@enron.com, jean.mrha@enron.com,...",[gasfundy.corp.enron.com],0
1,announcements.enron@enron.com,22-02-2002 07:30:36,Outlook Web Access Outage,Enron's external Outlook Web Access (OWA) (htt...,[dl-ga-all_domestic@enron.com],[mail.enron.com],0
2,perfmgmt@enron.com,06-11-2000 14:59:00,Year End 2000 Performance Feedback,NOTE: YOU WILL RECEIVE THIS MESSAGE EACH TIME ...,[jane.m.tholt@enron.com],[pep.corp.enron.com],0
3,jad.doan@enron.com,29-10-2001 18:11:54,TRV Notification: (CE P/L - 10/29/2001),"The report named: CE P/L , published as of 10/...","[brad.jones@enron.com, lisa.kinsey@enron.com, ...",[trv.corp.enron.com],0
4,enron.announcements@enron.com,05-06-2001 11:46:00,Managers Training,Attention supervisors and managers! =01; =01; ...,[all.houston@enron.com],[corptraining.com],0


In [117]:
df.count()

,0
From,31495
Date,31495
Subject,31495
Body,31495
Recipients,31495
Domains,31495
Label,31495


In [118]:
print(df.dtypes)

From          object
Date          object
Subject       object
Body          object
Recipients    object
Domains       object
Label          int64
dtype: object


In [119]:
print(type(df['Recipients'].iloc[0]))
print(type(df['Domains'].iloc[0]))

<class 'list'>
<class 'list'>


In [120]:
print(df.loc[273])

From                                   vince.kaminski@enron.com
Date                                        31-12-1979 16:00:00
Subject                                        Re: Full version
Body          Darrell, Thanks a lot. I really appreciate you...
Recipients                                [duffie@stanford.edu]
Domains                                          [stanford.edu]
Label                                                         0
Name: 273, dtype: object


In [121]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y %H:%M:%S', errors='coerce')

In [122]:
print(type(df['Date'].iloc[0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [123]:
print("Start date:", df['Date'].min())
print("End date:", df['Date'].max())

Start date: 1979-12-31 16:00:00
End date: 2002-07-11 10:05:00


In [124]:
df.count()

,0
From,31495
Date,31495
Subject,31495
Body,31495
Recipients,31495
Domains,31495
Label,31495


In [125]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y %H:%M:%S', errors='coerce')
df = df[df['Date'] >= pd.Timestamp('2000-01-01')]

In [126]:
df.count()

,0
From,31131
Date,31131
Subject,31131
Body,31131
Recipients,31131
Domains,31131
Label,31131


In [127]:
df['Recipients'] = df['Recipients'].apply(lambda x: tuple(x) if isinstance(x, list) else x)
df = df.drop_duplicates(subset=['From', 'Subject', 'Body', 'Recipients'])
df = df.reset_index(drop=True)

In [128]:
df.count()

,0
From,13055
Date,13055
Subject,13055
Body,13055
Recipients,13055
Domains,13055
Label,13055


In [129]:
df = df.reset_index(drop=True)

In [130]:
df['Recipients'] = df['Recipients'].apply(lambda x: list(x) if isinstance(x, tuple) else x)

In [131]:
df.head()

,From,Date,Subject,Body,Recipients,Domains,Label
0,kenny.soignet@enron.com,2000-12-06 05:06:00,AGA for 12/1/00 is -73,AGA for 12/1/00 is -73 Website information: ht...,"[kenny.soignet@enron.com, jean.mrha@enron.com,...",[gasfundy.corp.enron.com],0
1,announcements.enron@enron.com,2002-02-22 07:30:36,Outlook Web Access Outage,Enron's external Outlook Web Access (OWA) (htt...,[dl-ga-all_domestic@enron.com],[mail.enron.com],0
2,perfmgmt@enron.com,2000-11-06 14:59:00,Year End 2000 Performance Feedback,NOTE: YOU WILL RECEIVE THIS MESSAGE EACH TIME ...,[jane.m.tholt@enron.com],[pep.corp.enron.com],0
3,jad.doan@enron.com,2001-10-29 18:11:54,TRV Notification: (CE P/L - 10/29/2001),"The report named: CE P/L , published as of 10/...","[brad.jones@enron.com, lisa.kinsey@enron.com, ...",[trv.corp.enron.com],0
4,enron.announcements@enron.com,2001-06-05 11:46:00,Managers Training,Attention supervisors and managers! =01; =01; ...,[all.houston@enron.com],[corptraining.com],0


In [132]:
df.to_csv('cleaned_emails.csv', index=False)

In [133]:
top_senders = df['From'].value_counts().head(20)
print(top_senders)

From
vince.kaminski@enron.com         657
jeff.dasovich@enron.com          384
perfmgmt@enron.com               311
kay.mann@enron.com               276
enron.announcements@enron.com    248
rod.hayslett@enron.com           242
eric.bass@enron.com              241
richard.sanders@enron.com        178
j.kaminski@enron.com             171
michelle.cash@enron.com          169
tana.jones@enron.com             158
matthew.lenhart@enron.com        156
sara.shackleton@enron.com        151
chris.germany@enron.com          135
shelley.corman@enron.com         134
steven.kean@enron.com            123
m..scott@enron.com               116
d..steffes@enron.com             113
sylvia.hu@enron.com              109
louise.kitchen@enron.com         101
Name: count, dtype: int64


In [134]:
unique_senders = df['From'].nunique()
print(f"Unique senders: {unique_senders}")

Unique senders: 1567
